# Interactive Proofs: Schnorr протокол и Fiat-Shamir

В этом ноутбуке мы:
1. Реализуем Schnorr identification протокол
2. Запустим симуляцию Prover-Verifier
3. Покажем уменьшение вероятности обмана с ростом раундов
4. Применим Fiat-Shamir transform
5. Покажем что результат -- это Schnorr подпись (CRYPTO-12)

**Предварительные знания:** Цифровые подписи (CRYPTO-11/12), Schnorr подпись (CRYPTO-12), commitment schemes (ZK-02)

## 1. Setup: эллиптическая кривая

In [ ]:
from ecdsa import SECP256k1
import hashlib
import random
import os

# Параметры кривой secp256k1
G = SECP256k1.generator
order = SECP256k1.order

# Генерируем секретный ключ x
x = random.randint(1, order - 1)

# Вычисляем открытый ключ P = x * G
P = x * G

print("=== Setup ===")
print(f"Кривая: secp256k1")
print(f"Порядок: {order}")
print(f"Секретный ключ x: {x}")
print(f"Открытый ключ P = xG:")
print(f"  P.x = {P.x()}")
print(f"  P.y = {P.y()}")

## 2. Schnorr Identification Protocol

Три функции -- три хода Sigma протокола:
1. **Prover commit**: k = random, R = kG
2. **Verifier challenge**: c = random
3. **Prover respond**: s = k + c*x (mod n)
4. **Verify**: s*G == R + c*P?

In [ ]:
def prover_commit(G, order):
    """
    Шаг 1 (Prover): выбрать случайный k, вычислить R = kG.
    Возвращает (k, R).
    """
    k = random.randint(1, order - 1)
    R = k * G
    return k, R


def verifier_challenge(order):
    """
    Шаг 2 (Verifier): выбрать случайный challenge c.
    Возвращает c.
    """
    return random.randint(1, order - 1)


def prover_respond(k, c, x, order):
    """
    Шаг 3 (Prover): вычислить response s = k + c*x (mod order).
    Возвращает s.
    """
    return (k + c * x) % order


def verifier_check(G, P, R, c, s):
    """
    Шаг 4 (Verifier): проверить s*G == R + c*P.
    Возвращает True если проверка пройдена.
    """
    lhs = s * G       # s*G
    rhs = R + c * P   # R + c*P
    return lhs.x() == rhs.x() and lhs.y() == rhs.y()


# Один раунд протокола
print("=== Один раунд Schnorr Identification ===")
k, R = prover_commit(G, order)
c = verifier_challenge(order)
s = prover_respond(k, c, x, order)
valid = verifier_check(G, P, R, c, s)

print(f"Prover commit:   k = {k}")
print(f"                 R = kG, R.x = {R.x()}")
print(f"Verifier challenge: c = {c}")
print(f"Prover response: s = k + c*x mod n = {s}")
print(f"Verification:    s*G == R + c*P: {valid}")

## 3. Полный раунд с математикой

Разберём почему верификация работает:
```
s*G = (k + c*x)*G = k*G + c*x*G = R + c*P
```

In [ ]:
# Полный раунд с промежуточными вычислениями
k, R = prover_commit(G, order)
c = verifier_challenge(order)
s = prover_respond(k, c, x, order)

print("=== Математика верификации ===")
print(f"s = k + c*x (mod n)")
print(f"s = {k} + {c} * x (mod n) = {s}")
print()

# Левая часть: s*G
lhs = s * G
print(f"Левая часть: s*G")
print(f"  s*G.x = {lhs.x()}")
print()

# Правая часть пошагово: R + c*P
cP = c * P
rhs = R + cP
print(f"Правая часть: R + c*P")
print(f"  R = k*G, R.x = {R.x()}")
print(f"  c*P.x = {cP.x()}")
print(f"  (R + c*P).x = {rhs.x()}")
print()

match = lhs.x() == rhs.x() and lhs.y() == rhs.y()
print(f"s*G == R + c*P: {match}")
print()
print("Почему: s*G = (k + cx)*G = kG + cxG = R + cP")
assert match, "Верификация должна пройти!"

## 4. Честный Prover vs Cheater

Честный Prover знает x и ВСЕГДА проходит верификацию.
Cheater НЕ знает x и отвечает случайным s -- верификация почти всегда провалится.

In [ ]:
# Честный Prover: N раундов
N = 50
honest_pass = 0

for _ in range(N):
    k, R = prover_commit(G, order)
    c = verifier_challenge(order)
    s = prover_respond(k, c, x, order)  # Знает x!
    if verifier_check(G, P, R, c, s):
        honest_pass += 1

print(f"=== Честный Prover ({N} раундов) ===")
print(f"Пройдено: {honest_pass}/{N} ({honest_pass/N*100:.1f}%)")
print()

# Cheater: не знает x, выбирает случайный s
cheater_pass = 0

for _ in range(N):
    k, R = prover_commit(G, order)
    c = verifier_challenge(order)
    fake_s = random.randint(1, order - 1)  # НЕ знает x, случайный ответ
    if verifier_check(G, P, R, c, fake_s):
        cheater_pass += 1

print(f"=== Cheater ({N} раундов) ===")
print(f"Пройдено: {cheater_pass}/{N} ({cheater_pass/N*100:.1f}%)")
print()
print("Честный prover: ВСЕГДА проходит (completeness)")
print("Cheater: ~0% проходит (soundness)")

## 5. Вероятность обмана

Стратегия cheater: угадать c заранее, вычислить R = s*G - c*P для угаданного c.
Если угадал правильно -- верификация пройдёт. Если нет -- провалится.
Вероятность: 1/|challenge_space| за раунд. При большом challenge space ~0%.

In [ ]:
# Для наглядности используем маленькое пространство challenge
# (в реальности challenge_space = order ~ 2^256)

small_challenge_spaces = [2, 4, 8, 16]  # Размеры пространства challenge
simulations = 1000

print(f"Вероятность обмана при разном размере challenge space")
print(f"(стратегия: угадать c заранее)")
print(f"{'Challenge space':<20} {'Теория':<15} {'Эксперимент':<15}")
print("-" * 50)

for cs in small_challenge_spaces:
    success = 0
    for _ in range(simulations):
        # Cheater угадывает c
        c_guess = random.randint(0, cs - 1)
        # Verifier выбирает c
        c_actual = random.randint(0, cs - 1)
        if c_guess == c_actual:
            success += 1
    
    theory = 1 / cs
    experiment = success / simulations
    print(f"{cs:<20} {theory:<15.4f} {experiment:<15.4f}")

print()
print(f"Для secp256k1: challenge space = ~2^256")
print(f"Вероятность обмана за 1 раунд = 1/2^256 ~ 0")
print(f"\nЗа N раундов с challenge space = 2:")
for n_rounds in [1, 5, 10, 20, 40]:
    prob = (1/2) ** n_rounds
    print(f"  {n_rounds:>3} раундов: P(обман) = 1/2^{n_rounds} = {prob:.10f}")

## 6. Fiat-Shamir Transform

Ключевая идея: заменить **случайный challenge от Verifier** на **hash от публичных данных**.

c = H(statement || R) вместо c = random from Verifier

Результат: **неинтерактивное** доказательство. Prover отправляет (R, s) одним сообщением.

In [ ]:
def fiat_shamir_prove(G, P, x, order, statement: bytes):
    """
    Неинтерактивное доказательство знания x (Fiat-Shamir).
    statement -- публичное утверждение (контекст).
    Возвращает (R, s) -- доказательство.
    """
    # Шаг 1: commitment
    k = random.randint(1, order - 1)
    R = k * G
    
    # Шаг 2: challenge через hash (ВМЕСТО verifier)
    R_bytes = R.x().to_bytes(32, 'big') + R.y().to_bytes(32, 'big')
    c_hash = hashlib.sha256(statement + R_bytes).digest()
    c = int.from_bytes(c_hash, 'big') % order
    
    # Шаг 3: response
    s = (k + c * x) % order
    
    return R, s


def fiat_shamir_verify(G, P, R, s, order, statement: bytes):
    """
    Верификация неинтерактивного доказательства.
    Verifier вычисляет c самостоятельно из statement и R.
    Проверяет: s*G == R + c*P.
    """
    # Вычисляем challenge из hash
    R_bytes = R.x().to_bytes(32, 'big') + R.y().to_bytes(32, 'big')
    c_hash = hashlib.sha256(statement + R_bytes).digest()
    c = int.from_bytes(c_hash, 'big') % order
    
    # Проверяем: s*G == R + c*P
    lhs = s * G
    rhs = R + c * P
    return lhs.x() == rhs.x() and lhs.y() == rhs.y()


# Демонстрация
statement = b"I know the private key for public key P"

print("=== Fiat-Shamir (неинтерактивное доказательство) ===")
R_proof, s_proof = fiat_shamir_prove(G, P, x, order, statement)

print(f"Statement: {statement.decode()}")
print(f"Proof: (R, s)")
print(f"  R.x = {R_proof.x()}")
print(f"  s   = {s_proof}")
print()

# Верификация (Verifier может быть OFFLINE -- нужны только публичные данные)
valid = fiat_shamir_verify(G, P, R_proof, s_proof, order, statement)
print(f"Верификация: {valid}")
print()
print("Нет необходимости в живом Verifier!")
print("Prover отправляет (R, s) одним сообщением.")
print("Verifier в любое время вычисляет c = H(statement || R) и проверяет.")

## 7. Fiat-Shamir = Schnorr Подпись!

Когда statement = message (сообщение для подписи), Fiat-Shamir transform превращает
Schnorr identification protocol в **Schnorr signature scheme**:

- Подпись: (R, s) где c = H(message || R), s = k + c*x
- Верификация: c = H(message || R), s*G == R + c*P

Мы только что переизобрели Schnorr подпись из Module 2 (CRYPTO-12)!

In [ ]:
# Schnorr подпись через Fiat-Shamir
message = b"Transfer 1 BTC to Alice"

# Подписание = Fiat-Shamir prove с message как statement
R_sig, s_sig = fiat_shamir_prove(G, P, x, order, message)

print("=== Schnorr Подпись (через Fiat-Shamir) ===")
print(f"Message: {message.decode()}")
print(f"Public key P.x = {P.x()}")
print(f"Подпись (R, s):")
print(f"  R.x = {R_sig.x()}")
print(f"  s   = {s_sig}")
print()

# Верификация = Fiat-Shamir verify с message как statement
valid = fiat_shamir_verify(G, P, R_sig, s_sig, order, message)
print(f"Верификация подписи: {valid}")
print()

# Подмена сообщения
fake_message = b"Transfer 100 BTC to Eve"
fake_valid = fiat_shamir_verify(G, P, R_sig, s_sig, order, fake_message)
print(f"Подмена сообщения на '{fake_message.decode()}'")
print(f"Верификация поддельного сообщения: {fake_valid}")
print()

print("=" * 60)
print("ВЫВОД:")
print("  Schnorr identification + Fiat-Shamir = Schnorr signature")
print("  (R, s) -- подпись, c = H(message || R) -- challenge")
print("  Это ТОЖДЕСТВЕННО Schnorr подписи из CRYPTO-12!")
print()
print("  Интерактивный протокол -> Неинтерактивная подпись")
print("  Fiat-Shamir transform -- фундамент ВСЕХ SNARKs и STARKs")

## 8. Упражнения

### Упражнение 1: OR-proof

Реализуйте OR-proof: докажите знание x1 ИЛИ x2 (при P1=x1*G, P2=x2*G),
не раскрывая КАКОЙ именно секрет вы знаете.

Подсказка: зная x1, симулируйте второй proof (для P2) и используйте
c = c1 + c2 (mod order) чтобы связать оба proof.

In [ ]:
# Упражнение 1: Ваш код здесь
# Подсказка:
# 1. Prover знает x1 (для P1), не знает x2 (для P2)
# 2. Для P2: выбрать c2 заранее, вычислить R2 = s2*G - c2*P2
# 3. Для P1: честный commit R1 = k1*G
# 4. c = H(P1 || P2 || R1 || R2)
# 5. c1 = c - c2 (mod order)
# 6. s1 = k1 + c1*x1 (mod order)
# 7. Proof: (R1, R2, c1, c2, s1, s2)


### Упражнение 2: Batch Verification

Реализуйте batch verification для N Schnorr proofs.
Вместо N отдельных проверок s_i*G == R_i + c_i*P_i,
объедините в одну проверку с случайными весами alpha_i:

sum(alpha_i * s_i) * G == sum(alpha_i * R_i) + sum(alpha_i * c_i * P_i)

In [ ]:
# Упражнение 2: Ваш код здесь
# Подсказка:
# 1. Сгенерируйте N = 10 Fiat-Shamir proofs
# 2. Для batch: выберите alpha_i = random для каждого proof
# 3. lhs = sum(alpha_i * s_i) * G
# 4. rhs = sum(alpha_i * R_i) + sum(alpha_i * c_i) * P
# 5. Если ВСЕ proofs корректны: lhs == rhs
# 6. Сравните время: N отдельных vs 1 batch
